In [ ]:
# @title 🚀 全面升級數據整合平台 (v18) - 啟動器
#@markdown ---
#@markdown ### **核心架構：模組化、高穩定性、GitHub 直連**
#@markdown 歡迎使用數據整合平台 v18！此版本基於強化的模組化工程構建，提供前所未有的執行穩定性與更清晰的程式碼結構。
#@markdown - **智慧型程式碼更新**：啟動器會自動從您指定的 GitHub 儲存庫拉取最新程式碼。
#@markdown - **一站式參數配置**：所有可調整參數均集中於下方，方便您快速完成設定。
#@markdown - ✨ **功能更新**：執行完畢後，將在下方提供「一鍵複製全部日誌」按鈕，並採用全新深色外觀，解決中文亂碼問題。
#@markdown ---
#@markdown ### ⚙️ **執行核心設定**
#@markdown ---
#@markdown #### **(A) 系統與程式碼來源**
#@markdown 指定數據平台的核心程式碼來源。除非您是開發者或使用自訂版本，否則請保留預設值。
repo_url = "https://github.com/hsp1234-web/taifexd-date.git" #@param {type:"string"}
#@markdown ---
#@markdown #### **(B) 執行模式選擇**
#@markdown ✅ **Google Drive 整合模式**：勾選此項，您的專案資料夾與產出將同步並永久保存於您的 Google Drive。
#@markdown ❌ **本地臨時模式**：取消勾選，所有檔案將儲存於此 Colab 的臨時空間，適合快速測試，但關閉分頁後資料會遺失。
enable_google_drive_integration = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **(C) 主要路徑與檔案名稱**
#@markdown 設定您在 Google Drive 中的專案根目錄，以及輸出的資料庫與日誌檔名。
project_folder = "MyTaifexDataProject" #@param {type:"string"}
database_name = "processed_data.duckdb" #@param {type:"string"}
log_name = "pipeline.log" #@param {type:"string"}
#@markdown --- 
#@markdown #### **(D) 特定檔案處理**
#@markdown 若要指定處理單一或多個ZIP檔，請在此輸入檔名，並用逗號 (`,`) 分隔。留空則代表處理輸入目錄中的全部 ZIP 檔案。
target_zip_files = "" #@param {type:"string"}
#@markdown ---
#@markdown #### **(E) 開發者選項 (Developer)**
#@markdown ⚠️ **除錯模式**：僅在需要深入分析問題時開啟。將啟用詳細日誌與硬體效能監控，日誌檔案會變得非常大。
debug_mode = False #@param {type:"boolean"}

# ======================================================================
#         穩健執行區 (請勿修改此線以下內容)
# ======================================================================
import os
import sys
import shutil
import io
import contextlib
import base64
import subprocess
from google.colab import drive
from IPython.display import display, HTML

# --- 日誌捕捉與即時顯示的核心類別 ---
class OutputTee(io.StringIO):
    def __init__(self, original_stream):
        super().__init__()
        self._original_stream = original_stream
    def write(self, s):
        self._original_stream.write(s)
        super().write(s)
    def flush(self):
        self._original_stream.flush()
        super().flush()

# --- 主要執行邏輯 ---
log_capture = OutputTee(sys.stdout)
final_status = "✅ 正常結束"

with contextlib.redirect_stdout(log_capture), contextlib.redirect_stderr(log_capture):
    try:
        print("--- 參數動態顯示 (執行時更新) ---")
        print(f"程式碼來源: {repo_url}")
        print(f"Google Drive 整合模式: {'啟用' if enable_google_drive_integration else '停用 (本地測試模式)'}")
        print(f"專案資料夾: {project_folder}")
        print(f"輸出資料庫: {database_name}")
        print(f"輸出日誌檔: {log_name}")
        print(f"指定ZIP檔: {'全部' if not target_zip_files else target_zip_files}")
        print("--------------------\n")

        if enable_google_drive_integration:
            print("⏳ 正在請求掛載 Google Drive...")
            drive.mount('/content/drive', force_remount=True)
            print("✅ Google Drive 掛載成功！")
        else:
            print("ℹ️ Google Drive 整合已停用。所有操作將在本地臨時環境進行。")
        print("\n")

        print("⏳ 正在準備專案程式碼...")
        os.chdir('/content')
        repo_name = 'taifexd-date'
        repo_path = os.path.join('/content', repo_name)

        if os.path.exists(repo_path):
            print(f"  - 偵測到舊的 '{repo_path}' 目錄，正在刪除...")
            shutil.rmtree(repo_path)

        print(f"  - 執行 git clone，開始從 {repo_url} 下載專案...")
        subprocess.run(['git', 'clone', '-q', repo_url, repo_name], check=True)

        os.chdir(repo_path)
        print(f"✅ 專案程式碼準備完成！目前工作目錄: {os.getcwd()}\n")

        print("⏳ 正在同步 lock 檔案並安裝依賴...")
        subprocess.run(['pip', 'install', 'poetry', '-q'], check=True)
        print("  - 執行 poetry lock...")
        subprocess.run(['poetry', 'lock'], check=True, capture_output=True)
        print("  - 執行 poetry install...")
        subprocess.run(['poetry', 'install', '--no-root', '--without', 'dev'], check=True)
        print("✅ 所有 Python 套件均已安裝完成！\n")

        print("🚀 即將啟動數據整合平台主程式...")
        command_parts = [
            "poetry", "run", "python", "main.py",
            f"--project-folder-name={project_folder}",
            f"--database-name={database_name}",
            f"--log-name={log_name}",
            f"--zip-files={target_zip_files}"
        ]
        if not enable_google_drive_integration:
            command_parts.append("--no-gdrive")

        if debug_mode:
            command_parts.append("--debug")

        print(f"  - 將執行的最終指令:\n    {' '.join(command_parts)}")
        print("-" * 20 + " 程式開始執行 " + "-" * 20)

        subprocess.run(command_parts, check=True)

        print("-" * 20 + " 程式執行完畢 " + "-" * 20)

    except subprocess.CalledProcessError as e:
        final_status = f"❌ 子程序執行失敗 (返回碼 {e.returncode})。請檢查上方日誌以了解詳細錯誤。"
        print(f"\n{final_status}")
    except Exception as e:
        final_status = f"❌ 發生未預期的錯誤：{e}"
        print(f"\n{final_status}")

# --- 產出最終結果與複製按鈕 ---
full_log_content = log_capture.getvalue()
encoded_logs = base64.b64encode(full_log_content.encode('utf-8')).decode('utf-8')

html_output = f""\
<div style=\"border: 2px solid #66FF66; padding: 15px; margin-top: 20px; border-radius: 8px; background-color: #1E1E1E; color: #E0E0E0; font-family: 'Monaco', 'Menlo', 'monospace';\">\
    <h3 style='color: #FFFFFF; margin-top:0; border-bottom: 1px solid #444; padding-bottom: 10px;'>📋 執行結果摘要</h3>\
    <p><strong>最終狀態：</strong><span style=\"color: {'#FF6B6B' if '❌' in final_status else '#66FF66'};\">{final_status}</span></p>\
    <button \
        id=\"copy-button\"\
        onclick=\"copyLogsToClipboard()\" \
        style=\"background-color: #333; color: white; padding: 10px 15px; border: 1px solid #66FF66; border-radius: 5px; cursor: pointer; font-size: 14px; transition: background-color 0.2s;\"\
    >\
        一鍵複製上方完整日誌\
    </button>\
    <p id=\"copy-status\" style='font-size: 12px; color: #999; margin-top: 8px; height: 16px; transition: color 0.3s;'></p>\
</div>\
<textarea id=\"hidden-logs\" style=\"opacity: 0; position: absolute; left: -9999px; width: 0; height: 0;\"></textarea>\
<script>\
  (function() {{ \
    try {{ \
      const base64Data = '{encoded_logs}';\
      const binaryString = atob(base64Data);\
      const bytes = new Uint8Array(binaryString.length);\
      for (let i = 0; i < binaryString.length; i++) {{ \
          bytes[i] = binaryString.charCodeAt(i);\
      }} \
      const decodedString = new TextDecoder('utf-8').decode(bytes);\
      document.getElementById('hidden-logs').value = decodedString;\
    }} catch (e) {{ \
        console.error('Base64 解碼失敗:', e);\
        document.getElementById('hidden-logs').value = '日誌內容解碼失敗，可能包含特殊字元。';\
    }} \
  }})();\
  \
  function copyLogsToClipboard() {{ \
    const textarea = document.getElementById('hidden-logs');\
    const status_p = document.getElementById('copy-status');\
    const button = document.getElementById('copy-button');\
    textarea.select();\
    textarea.setSelectionRange(0, 999999);\
    try {{ \
      const successful = document.execCommand('copy');\
      if (successful) {{ \
        status_p.textContent = '✅ 已成功複製到剪貼簿！';\
        status_p.style.color = '#66FF66';\
        button.style.backgroundColor = '#2E7D32';\
      }} else {{ \
        status_p.textContent = '❌ 複製失敗。您的瀏覽器可能不支援此操作。';\
        status_p.style.color = '#FF6B6B';\
      }} \
    }} catch (err) {{ \
      status_p.textContent = '❌ 複製時發生錯誤，請手動複製。';\
      status_p.style.color = '#FF6B6B';\
      console.error('複製日誌失敗: ', err);\
    }} \
    setTimeout(() => {{ \
        status_p.textContent = '';\
        button.style.backgroundColor = '#333';\
    }}, 3000);\
  }} \
</script>\
"""

display(HTML(html_output))
